# Traditional ETL Prototype
Using NYC Taxi Trip Data Sample

In [ ]:
import pandas as pd
import numpy as np


In [2]:
# Load the 10K-row sample NYC Taxi data
df = pd.read_csv('../data/processed/yellow_tripdata_sample_10k.csv')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-29 17:52:02,2023-01-29 17:56:43,1.0,1.17,1.0,N,262,74,2,7.2,0.0,0.5,0.00,0.0,1.0,11.20,2.5,0.0
1,1,2023-01-08 15:57:24,2023-01-08 16:02:47,1.0,0.90,1.0,N,229,237,2,6.5,2.5,0.5,0.00,0.0,1.0,10.50,2.5,0.0
2,2,2023-01-21 19:38:01,2023-01-21 19:45:02,1.0,0.95,1.0,N,45,261,1,7.9,0.0,0.5,2.38,0.0,1.0,14.28,2.5,0.0
3,2,2023-01-23 16:07:31,2023-01-23 16:26:46,5.0,0.88,1.0,N,237,141,1,16.3,2.5,0.5,1.50,0.0,1.0,24.30,2.5,0.0
4,2,2023-01-26 21:21:08,2023-01-26 21:24:48,2.0,1.03,1.0,N,229,140,1,6.5,1.0,0.5,2.30,0.0,1.0,13.80,2.5,0.0


In [3]:
# Map payment_type codes to human-readable labels
payment_map = {
    1: 'Credit Card',
    2: 'Cash',
    3: 'No Charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided Trip'
}
df['payment_label'] = df['payment_type'].map(payment_map)
df[['payment_type', 'payment_label']].drop_duplicates()

,payment_type,payment_label
0,2,Cash
2,1,Credit Card
13,0,NaN
48,3,No Charge
102,4,Dispute


In [4]:
# Calculate trip duration in minutes
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['trip_duration_min'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
df[['trip_duration_min']].describe()

,trip_duration_min
count,10000.000000
mean,15.216973
std,36.207035
min,0.000000
25%,7.116667
50%,11.400000
75%,18.100000
max,1437.733333


In [6]:
from memory_profiler import memory_usage
import time

In [7]:
def transform_etl(df):
    payment_map = {
        1: 'Credit Card',
        2: 'Cash',
        3: 'No Charge',
        4: 'Dispute',
        5: 'Unknown',
        6: 'Voided Trip'
    }
    df['payment_label'] = df['payment_type'].map(payment_map)
    df['trip_duration_min'] = (
        pd.to_datetime(df['tpep_dropoff_datetime']) - pd.to_datetime(df['tpep_pickup_datetime'])
    ).dt.total_seconds() / 60
    return df

In [8]:
start_time = time.time()
mem_usage = memory_usage((transform_etl, (df.copy(),)))
elapsed_time = time.time() - start_time

print(f"Memory usage: {max(mem_usage) - min(mem_usage):.2f} MB")
print(f"Execution time: {elapsed_time:.4f} seconds")

Memory usage: 37.14 MB
Execution time: 0.8799 seconds
